In [1]:
#import zipfile
import numpy as np
import pandas
import os
import io
from pprint import pprint
import json
from datetime import datetime
print('imported')

imported


In [2]:
ladj_games = []
badj_games = []
padj_games = []

for i in range(1921,2018):
#for i in range(1921,1924):
    print('Year {0:}'.format(i))
    thisyearfiles = []

    # import list of teams from retrosheet
    teams_df = pandas.read_csv('teams_all.csv')
    teams_df['firstGame'] = pandas.to_datetime(teams_df['firstGame'], errors='coerce')
    teams_df['lastGame'] = pandas.to_datetime(teams_df['lastGame'], errors='coerce')

    teams_df['firstYear'] = pandas.to_numeric(teams_df['firstGame'].apply(lambda x: x.year), errors='coerce')
    teams_df['lastYear'] = pandas.to_numeric(teams_df['lastGame'].apply(lambda x: x.year), errors='coerce')
    teams_df['lastYear'] = teams_df['lastYear'].fillna(2017)

    teams_df['firstYear'] = pandas.to_numeric(teams_df['firstYear'], downcast='integer', errors='coerce')
    teams_df['lastYear'] = pandas.to_numeric(teams_df['lastYear'], downcast='integer', errors='coerce')

    teams_df = teams_df.fillna('')
    #print('This year\'s teams found...')

    teams_this_year_df = teams_df[(teams_df['firstYear'] <= i) & (teams_df['lastYear'] >= i)]
    for oneindex, oneteam in teams_this_year_df.iterrows():
        oneteamname = oneteam['teamID']
        itsleague = oneteam['league'][0:1]
        afile = '{:.0f}'.format(i)+oneteamname+'.EV'+itsleague
        if (afile in (os.listdir('events/'))):
            thisyearfiles.append('events/'+afile)
    #print('{0:.0f}: found {1:,.0f} files...'.format(i, len(thisyearfiles)))
    allgames = []
    gamecount = -1  # MUST START AT -1
    for filename in thisyearfiles:
        #print('Parsing file: {0:}...'.format(filename))
        for line in io.open(filename,encoding='utf-8').readlines():
            linearray = line.split(',')
            linearray[-1] = linearray[-1][0:-1]  # cut off newline character
            if (linearray[0] == 'id'):
                # 'id' starts the next game, so immediately advance gamecount  
                # record prior game data by adding it to the gamelogs list, unless gamecount = 0 in which case no data yet
                if (gamecount >= 0):
                    thisgamedict['info'] = info_dict
                    thisgamedict['visiting_starters'] = vis_start_list
                    thisgamedict['home_starters'] = home_start_list
                    thisgamedict['events'] = events
                    thisgamedict['data'] = data_list
                    # append the dictionary describing this game to the list of games
                    allgames.append(thisgamedict)
                # increment game count
                gamecount = gamecount + 1
                # initialize new dictionaries to hold this game's info
                #print('Initializing game {0:,.0f}...'.format(gamecount))
                thisgamedict = {}
                info_dict = {}
                start_dict = {}
                vis_start_list = []
                home_start_list = []
                events = []
                eventnum = 1
                data_list = []
                thisgamedict['id'] = linearray[1]
            elif (linearray[0] == 'version'):
                thisgamedict['version'] = linearray[1]
            elif (linearray[0] == 'info'):
                info_dict[linearray[1]] = linearray[2]
            elif (linearray[0] == 'start'):
                thisbatter = {}
                thisbatter['player_id'] = linearray[1]
                #thisbatter['batter_name'] = linearray[2]
                thisbatter['order'] = linearray[4]
                thisbatter['position'] = linearray[5]
                if (linearray[3] == '0'):
                    vis_start_list.append(thisbatter)
                elif (linearray[3] == '1'):
                    home_start_list.append(thisbatter)
                else:
                    print('Neither visitor nor home!!!!')
            elif (linearray[0] == 'data'):
                thisdata = {}
                thisdata['type'] = linearray[1]
                thisdata['player'] = linearray[2]
                thisdata['thevalue'] = linearray[3]
                data_list.append(thisdata)
            else:
                thisevent = {}
                thisevent['event_in_game'] = eventnum
                thisevent['event_type'] = linearray[0]            
                if (thisevent['event_type'] == 'com'):
                    thisevent['comment'] = linearray[1]
                elif (thisevent['event_type'] == 'sub'):
                    thisevent['sub_id'] = linearray[1]
                    thisevent['sub_team'] = linearray[3]
                    thisevent['sub_name'] = linearray[2]
                    thisevent['sub_batting_order'] = linearray[4]
                    thisevent['sub_position'] = linearray[5]
                    if (thisevent['sub_team'] not in ('0','1')):
                        print('weird sub')
                        pprint(thisgamedict['id'])                 
                        pprint(linearray)      
                        if (
                            ((thisgamedict['id'] == 'BSN192208070') & (linearray[1] == 'barfc101'))
                            | ((thisgamedict['id'] == 'NY1194906010') & (linearray[1] == 'burgs101'))
                        ):                    
                            thisevent['sub_team'] = linearray[4]
                            thisevent['sub_name'] = linearray[3] + ' ' + linearray[2]
                            thisevent['sub_batting_order'] = linearray[5]
                            thisevent['sub_position'] = linearray[6]
                        if (
                            ((thisgamedict['id'] == 'CIN193204240') & (linearray[1] == 'watkg101'))
                            | ((thisgamedict['id'] == 'SLN194408190') & (linearray[1] == 'pylee101'))
                            | ((thisgamedict['id'] == 'CLE196509100') & (linearray[1] == 'scher101'))
                        ):
                            thisevent['sub_team'] = linearray[4]
                            thisevent['sub_batting_order'] = linearray[5]
                            thisevent['sub_position'] = linearray[6]
                        if ((thisgamedict['id'] == 'SLA194205290') & (linearray[1] == 'trotb101')):
                            thisevent['sub_team'] = linearray[3][-1] 
                elif (thisevent['event_type'] == 'play'):
        #                print(linearray)
                        thisevent['inning'] = linearray[1]
                        thisevent['visitor_or_home'] = linearray[2]
                        thisevent['batter_id'] = linearray[3]
                        thisevent['pitch_count'] = linearray[4]
                        thisevent['pitch_sequence'] = linearray[5]
                        thisevent['event'] = linearray[6]
                elif (thisevent['event_type'] == 'badj'):
                    badj_games.append(thisgamedict['id'])
                    thisevent['adjusting_batter_id'] = linearray[1]
                    thisevent['adjusting_batter_hand'] = linearray[2]
    #                    pprint(thisevent)
                elif (linearray[0] == 'padj'):
                    padj_games.append(thisgamedict['id'])
                    thisevent['adjusting_pitcher_id'] = linearray[1]
                    thisevent['adjusting_pitcher_hand'] = linearray[2]
    #                    pprint(thisevent)
                elif (linearray[0] == 'ladj'):
                    ladj_games.append(thisgamedict['id'])
    #                    print(linearray)
                    thisevent['visitor_or_home'] = linearray[1]
                    thisevent['actual_order_of_wrong_batter'] = linearray[2]
    #                    pprint(thisevent)
                else:
                    print(linearray)
                events.append(thisevent)
                eventnum = eventnum + 1
                
        # add data for the final game, which we missed above
        thisgamedict['info'] = info_dict
        thisgamedict['visiting_starters'] = vis_start_list
        thisgamedict['home_starters'] = home_start_list
        thisgamedict['events'] = events
        thisgamedict['data'] = data_list

        allgames.append(thisgamedict)
        
    print('Processed {0:,.0f} games...'.format(len(allgames)))

    outfilename = '/home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_{0:}.json'.format(i)
    with open(outfilename, 'w') as f:
        json.dump(allgames, f, indent=2)
    print('File written: {0:}!\n'.format(outfilename))
print('Done!')

Year 1921
Processed 983 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1921.json!

Year 1922
weird sub
'BSN192208070'
['sub', 'barfc101', '"Clyde', 'Barfoot"', '0', '9', '1']
Processed 993 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1922.json!

Year 1923
Processed 1,007 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1923.json!

Year 1924
Processed 1,024 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1924.json!

Year 1925
Processed 1,014 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1925.json!

Year 1926
Processed 1,023 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1926.json!

Year 1927
Processed 1,053 games...

File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1973.json!

Year 1974
Processed 1,968 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1974.json!

Year 1975
Processed 1,957 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1975.json!

Year 1976
Processed 1,962 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1976.json!

Year 1977
Processed 2,128 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1977.json!

Year 1978
Processed 2,127 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1978.json!

Year 1979
Processed 2,123 games...
File written: /home/idies/workspace/Temporary/raddick/baseball_scratch/events_raw_by_year/events_raw_1979.json!


In [3]:
ldf = pandas.DataFrame(ladj_games)
ldf = ldf.drop_duplicates()
ldf.columns = ['gameID']
ldf = ldf.set_index('gameID')

bdf = pandas.DataFrame(badj_games)
bdf = bdf.drop_duplicates()
bdf.columns = ['gameID']
bdf = bdf.set_index('gameID')

pdf = pandas.DataFrame(padj_games)
pdf = pdf.drop_duplicates()
pdf.columns = ['gameID']
pdf = pdf.set_index('gameID')

ldf.to_csv('/home/idies/workspace/Temporary/raddick/baseball_scratch/ladj.csv')
bdf.to_csv('/home/idies/workspace/Temporary/raddick/baseball_scratch/badj.csv')
pdf.to_csv('/home/idies/workspace/Temporary/raddick/baseball_scratch/padj.csv')

print('Done')

Done
